In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb

In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


In [3]:
def id_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    id_u2c(q, qdot, qddot)
    
def id_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    id_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, qddot)
    
def id_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, id_rbdl)
    

    
    
def C_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_u2c(q, qdot)
    
def C_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, zeros_pb)
    
def C_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, zeros_rbdl, id_rbdl)

def C_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, C_kdl)
    
    
    

def M_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_u2c(q)
    
def M_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToMass(q_kdl, M_kdl)

def M_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_pb = pb.calculateMassMatrix(pbmodel, q)
    
def M_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, M_rbdl)

    


    
def g_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_u2c(q)
    
def g_pb_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    g_pb = pb.calculateInverseDynamics(pbmodel, q, zeros_pb, zeros_pb)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, zeros_rbdl, zeros_rbdl, id_rbdl)

def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)
    
    
    
    

def fd_aba_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_u2c(q, qdot, tau)
    

def fd_crba_u2c_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_u2c(q, qdot, tau)
    
def fd_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_np, qddot_np)

In [4]:
sim = pb.connect(pb.DIRECT)

#storage for timing data
mediantime_id_u2c = 0
mediantime_M_u2c = 0
mediantime_C_u2c = 0
mediantime_g_u2c = 0
mediantime_fd_crba_u2c = 0
mediantime_fd_aba_u2c = 0


mediantime_id_rbdl = 0
mediantime_g_rbdl = 0
mediantime_C_rbdl = 0
mediantime_fd_aba_rbdl = 0

mediantime_M_kdl = 0
mediantime_g_kdl = 0
mediantime_C_kdl = 0

mediantime_id_pb = 0
mediantime_g_pb = 0
mediantime_C_pb = 0
mediantime_M_pb = 0

nitr = 10000
nrepeat = 100

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/ur5_mod.urdf"
root = 'base_link'
tip = "tool0"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
robot = u2c.URDFparser()
robot_desc = robot.from_file(path_to_urdf)

jlist, names, qmax, qmin = robot.get_joint_info(root, tip)
njoints = robot.get_n_joints(root, tip)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
tau_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
id_rbdl = np.zeros(njoints)
C_rbdl = np.zeros(njoints)
zeros_rbdl = np.zeros(njoints)


#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)
C_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q = [None]*njoints
qdot = [None]*njoints
qddot = [None]*njoints
tau = [None]*njoints
g_pb = [None]*njoints
M_pb = [None]*njoints
id_pb = [None]*njoints

zeros_pb = []
for i in range(njoints):
    zeros_pb.append(0.)

                  
M_kdl = kdl.JntSpaceInertiaMatrix(njoints)
M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)

gravity_u2c = [0, 0, -9.81]
C_u2c = robot.get_coriolis_rnea(root, tip)
g_u2c = robot.get_gravity_rnea(root, tip, gravity_u2c)
M_u2c = robot.get_inertia_matrix_crba(root, tip)
id_u2c = robot.get_inverse_dynamics_rnea(root, tip, gravity_u2c)
fd_aba_u2c = robot.get_forward_dynamics_aba(root, tip, gravity_u2c)
fd_crba_u2c = robot.get_forward_dynamics_crba(root, tip, gravity_u2c)

#sim = pb.disconnect(pb.DIRECT)

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/urdf/ur5_mod.urdf


In [ ]:
ur5_timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c = median(ur5_timeit_g_u2c)/nitr

ur5_timeit_C_u2c = repeat("C_u2c_func()", setup = "from __main__ import C_u2c_func", repeat = nrepeat, number = nitr)
mediantime_C_u2c = median(ur5_timeit_C_u2c)/nitr

ur5_timeit_M_u2c = repeat("M_u2c_func()", setup = "from __main__ import M_u2c_func", repeat = nrepeat, number = nitr)
mediantime_M_u2c = median(ur5_timeit_M_u2c)/nitr

ur5_timeit_id_u2c = repeat("id_u2c_func()", setup = "from __main__ import id_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c = median(ur5_timeit_id_u2c)/nitr

ur5_timeit_fd_crba_u2c = repeat("fd_crba_u2c_func()", setup = "from __main__ import fd_crba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fd_crba_u2c = median(ur5_timeit_fd_crba_u2c)/nitr

ur5_timeit_fd_aba_u2c = repeat("fd_aba_u2c_func()", setup = "from __main__ import fd_aba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fd_aba_u2c = median(ur5_timeit_fd_aba_u2c)/nitr

print "\nur5_g_u2c=", mediantime_g_u2c
print "\nur5_id_u2c=", mediantime_id_u2c
print "\nur5_C_u2c=", mediantime_C_u2c
print "\nur5_M_u2c=", mediantime_M_u2c
print "\nur5_fd_aba u2c=", mediantime_fd_aba_u2c
print "\nur5_fd_crba u2c=", mediantime_fd_crba_u2c

In [6]:
%store kuka_timeit_g_u2c
%store kuka_timeit_C_u2c
%store kuka_timeit_id_u2c
%store kuka_timeit_M_u2c
%store kuka_timeit_fd_crba_u2c
%store kuka_timeit_fd_aba_u2c


%store mediantime_g_u2c
%store mediantime_C_u2c
%store mediantime_id_u2c
%store mediantime_M_u2c
%store mediantime_fd_aba_u2c
%store mediantime_fd_crba_u2c

Stored 'kuka_timeit_g_u2c' (list)
Stored 'kuka_timeit_C_u2c' (list)
Stored 'kuka_timeit_id_u2c' (list)
Stored 'kuka_timeit_M_u2c' (list)
Stored 'kuka_timeit_fd_crba_u2c' (list)
Stored 'kuka_timeit_fd_aba_u2c' (list)
Stored 'mediantime_g_u2c' (float)
Stored 'mediantime_C_u2c' (float)
Stored 'mediantime_id_u2c' (float)
Stored 'mediantime_M_u2c' (float)
Stored 'mediantime_fd_aba_u2c' (float)
Stored 'mediantime_fd_crba_u2c' (float)


In [5]:
ur5_timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(ur5_timeit_g_pb)/nitr

ur5_timeit_C_pb = repeat("C_pb_func()", setup = "from __main__ import C_pb_func", repeat = nrepeat, number = nitr)
mediantime_C_pb = median(ur5_timeit_C_pb)/nitr

ur5_timeit_id_pb = repeat("id_pb_func()", setup = "from __main__ import id_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(ur5_timeit_id_pb)/nitr

ur5_timeit_M_pb = repeat("M_pb_func()", setup = "from __main__ import M_pb_func", repeat = nrepeat, number = nitr)
mediantime_M_pb = median(ur5_timeit_M_pb)/nitr

print "\nur5_g_pb=", mediantime_g_pb
print "\nur5_id_pb=", mediantime_id_pb
print "\nur5_C_pb=", mediantime_C_pb
print "\nur5_M_pb=", mediantime_M_pb


ur5_g_pb= 5.51379919052e-06

ur5_id_pb= 1.05975031853e-05

ur5_C_pb= 8.02295207977e-06

ur5_M_pb= 5.73530197144e-06


In [8]:
%store kuka_timeit_g_pb
%store kuka_timeit_C_pb
%store kuka_timeit_id_pb
%store kuka_timeit_M_pb

Stored 'kuka_timeit_g_pb' (list)
Stored 'kuka_timeit_C_pb' (list)
Stored 'kuka_timeit_id_pb' (list)
Stored 'kuka_timeit_M_pb' (list)


In [6]:
ur5_timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(ur5_timeit_g_kdl)/nitr

ur5_timeit_C_kdl = repeat("C_kdl_func()", setup = "from __main__ import C_kdl_func", repeat = nrepeat, number = nitr)
mediantime_C_kdl = median(ur5_timeit_C_kdl)/nitr

ur5_timeit_M_kdl = repeat("M_kdl_func()", setup = "from __main__ import M_kdl_func", repeat = nrepeat, number = nitr)
mediantime_M_kdl = median(ur5_timeit_M_kdl)/nitr

print "\nur5_g_kdl=", mediantime_g_kdl
print "\nur5_C_kdl=", mediantime_C_kdl
print "\nur5_M_kdl=", mediantime_M_kdl


ur5_g_kdl= 1.28314495087e-05

ur5_C_kdl= 1.61038041115e-05

ur5_M_kdl= 1.35389924049e-05


In [10]:
%store kuka_timeit_g_kdl
%store kuka_timeit_C_kdl
%store kuka_timeit_M_kdl

Stored 'kuka_timeit_g_kdl' (list)
Stored 'kuka_timeit_C_kdl' (list)
Stored 'kuka_timeit_M_kdl' (list)


In [7]:


ur5_timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(ur5_timeit_g_rbdl)/nitr

ur5_timeit_C_rbdl = repeat("C_rbdl_func()", setup = "from __main__ import C_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_C_rbdl = median(ur5_timeit_C_rbdl)/nitr

ur5_timeit_id_rbdl = repeat("id_rbdl_func()", setup = "from __main__ import id_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(ur5_timeit_id_rbdl)/nitr

ur5_timeit_M_rbdl = repeat("M_rbdl_func()", setup = "from __main__ import M_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_M_rbdl = median(ur5_timeit_M_rbdl)/nitr

ur5_timeit_fd_rbdl = repeat("fd_rbdl_func()", setup = "from __main__ import fd_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_fd_rbdl = median(ur5_timeit_fd_rbdl)/nitr

print "\nur5_g_rbdl=", mediantime_g_rbdl
print "\nur5_id_rbdl=", mediantime_id_rbdl
print "\nur5_C_rbdl=", mediantime_C_rbdl
print "\nur5_M_rbdl=", mediantime_M_rbdl
print "\nur5_fd_rbdl=", mediantime_fd_rbdl


ur5_g_rbdl= 6.50115013123e-06

ur5_id_rbdl= 1.22493028641e-05

ur5_C_rbdl= 9.48075056076e-06

ur5_M_rbdl= 5.3733587265e-06

ur5_fd_rbdl= 1.31744503975e-05


In [12]:
%store kuka_timeit_g_rbdl
%store kuka_timeit_C_rbdl
%store kuka_timeit_id_rbdl
%store kuka_timeit_M_rbdl
%store kuka_timeit_fd_rbdl


Stored 'kuka_timeit_g_rbdl' (list)
Stored 'kuka_timeit_C_rbdl' (list)
Stored 'kuka_timeit_id_rbdl' (list)
Stored 'kuka_timeit_M_rbdl' (list)
Stored 'kuka_timeit_fd_rbdl' (list)
